**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
name = 'bpi_2012'
#name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process'  

parser = {
    'train': True,
    'test': True,
    'model_class': 'AE',
    'model_name': '',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.35,
    'input_dir': '../input/{}/'.format(name), 
#    'output_dir': './output/{}/'.format(name),
    'batch_size' : 16,
    'epochs' : 50,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 1000,
    'layer2': 100,
    'lr': 0.002,
    'betas': (0.9, 0.999),   
    'lr_decay': 0.90,
}

args = argparse.Namespace(**parser)
args.output_dir = './output/{0}_{1}/'.format(name, args.model_class)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

## Validate and test

In [12]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [13]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [14]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [15]:
missing_true_val.shape, missing_true_test.shape

((55671, 5), (47823, 5))

In [16]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [17]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [18]:
pd.isnull(normalized_missing_df).sum()

CaseID                   0
Activity             91643
CompleteTimestamp    91897
CumTimeInterval      91897
NormalizedTime       91897
dtype: int64

In [19]:
pd.isnull(missing_true_val).sum()

CaseID                   0
Activity             19324
CompleteTimestamp    19475
CumTimeInterval      19475
NormalizedTime       19475
dtype: int64

In [20]:
pd.isnull(missing_true_test).sum()

CaseID                   0
Activity             16787
CompleteTimestamp    16864
CumTimeInterval      16864
NormalizedTime       16864
dtype: int64

# Build model

## Define model

In [21]:
if args.model_class == 'AE':
    model = AE(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2)

if args.cuda:
    model.cuda()

## Define loss

In [22]:
# Define loss

def loss_function(recon_x, x, avai_mask):
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False) 
    return BCE

In [23]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [24]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
#lambda1 = lambda epoch: args.lr_decay ** epoch
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [25]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [26]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [27]:
def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, nan_matrix_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    nan_matrix_val = Variable(torch.Tensor(nan_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        nan_matrix_val = nan_matrix_val.cuda()
        
    recon_data = model(m_val)
        
    loss = loss_function(recon_data, c_val, nan_matrix_val)
    return loss.data[0]/missing_matrix_w_normalized_time_val.shape[0]

```python
def val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
       pad_matrix_val, cols_w_normalized_time, val_row_num,
       nan_time_index_val, nan_activity_index_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        
    recon_val = model(m_val)
    
    recon_df_w_normalized_time = convert2df(recon_val, pad_matrix_val, cols_w_normalized_time, val_row_num)
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_val, min_max_storage)
    submission_df = getSubmission(recon_df_w_time, missing_true_val, complete_true_val, first_timestamp)
    
    #evaluate
    time, acc = evaluation(submission_df, nan_time_index_val, nan_activity_index_val)
    
    
    return time/86400+1/acc
```

In [28]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,NaN,2012-01-05 06:57:15.554,8291911.008,0.000000e+00
1,7853,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:57:15.858,8291911.312,7.624972e-03
2,7853,A_DECLINED-COMPLETE,2012-01-05 06:57:55.323,8291950.777,9.974918e-01
3,7854,NaN,2012-01-05 06:59:14.543,8292029.997,0.000000e+00
4,7854,NaN,2012-01-05 06:59:15.103,8292030.557,5.115717e-07


In [29]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,2012-01-05 06:57:15.554,8291911.008,0.000000e+00
1,7853,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:57:15.858,8291911.312,7.624972e-03
2,7853,A_DECLINED-COMPLETE,2012-01-05 06:57:55.323,8291950.777,9.974918e-01
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.000000e+00
4,7854,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:59:15.103,8292030.557,5.115717e-07


# Train model

In [30]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader, 
                                                                complete_matrix_w_normalized_time_trainLoader,
                                                                avai_matrix_trainLoader)):
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())

        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()

            
        optimizer.zero_grad()
        
        recon_data = model(m_data)
        
        loss = loss_function(recon_data, c_data, avai_mask)
        
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [31]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        #scheduler.step()
        train_loss = train(epoch, model, optimizer)
        #val_score = val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
        #                    pad_matrix_val, cols_w_normalized_time, val_row_num, 
        #                    nan_time_index_val, nan_activity_index_val)
        val_score = val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val)
        
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
        
        #method 2 scheduler
        scheduler.step(val_score)
        
        end = time.time()
        print('====> Epoch {} | Time: {} ms| Train loss: {:.4f} | Val score: {:.4f}'.format(
                          epoch, (end-init)*1000 ,train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Time: 5696.900367736816 ms| Train loss: 36.7815 | Val score: 28.3500
====> Epoch 2 | Time: 5605.795621871948 ms| Train loss: 21.5948 | Val score: 25.3664
====> Epoch 3 | Time: 5585.98256111145 ms| Train loss: 18.3551 | Val score: 24.4870
====> Epoch 4 | Time: 5600.821018218994 ms| Train loss: 16.6892 | Val score: 24.3052
====> Epoch 5 | Time: 5594.825506210327 ms| Train loss: 16.2479 | Val score: 23.9544
====> Epoch 6 | Time: 5553.323268890381 ms| Train loss: 14.9025 | Val score: 26.2287
====> Epoch 7 | Time: 5567.888259887695 ms| Train loss: 14.1948 | Val score: 25.8368
====> Epoch 8 | Time: 5580.270767211914 ms| Train loss: 13.5452 | Val score: 23.9441
====> Epoch 9 | Time: 5567.224025726318 ms| Train loss: 13.1395 | Val score: 25.4938
====> Epoch 10 | Time: 5557.24573135376 ms| Train loss: 13.4128 | Val score: 25.5394
====> Epoch 11 | Time: 5567.283630371094 ms| Train loss: 12.8036 | Val score: 24.9518
====> Epoch 12 | Time: 5586.0631465911865 ms| Train loss: 11.6575

Process Process-131:
Process Process-132:
Process Process-130:
Process Process-127:
Process Process-129:
Process Process-128:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/pllab/miniconda2/envs/pydata/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/pllab/miniconda2/envs/pydata/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/pllab/miniconda2/envs/pydata/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/pllab/miniconda2/envs/pydata/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/pllab/miniconda2/envs/pydata/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pllab/minicond

KeyboardInterrupt: 

# Predict and evaluate

In [ ]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
    
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    print('\n')
    
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

In [ ]:
submission_df.head(10)

In [ ]:
submission.head(10)

In [ ]:
#bpi2013
#7.6
#75%

In [ ]:
#bpi2012
#0.59
#75%

In [ ]:
#helpdesk
#3.9
#85